In [1]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import math

import argparse
import cv2

# from boundbox import BoundBox

In /home/fizzer/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/fizzer/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/fizzer/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [ ]:
# when the robot detects it, it would move towards the arrow, stop, take a picture and determine direction?

## Opening Up the Camera

In [ ]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    cv2.imshow('Annotated Feed', frame)

    c = cv2.waitKey(1) # adds delay in feed so we can read 
    if c == 27: # this is the esc key
        break

cap.release()
cv2.destroyAllWindows()

## Detecting the Outline of the Arrow

Displays only the point (all vertical and horizontal lines filtered out). Shows up in red on the final annotated feed.

In [3]:
# code help from https://docs.opencv.org/3.4/d9/db0/tutorial_hough_lines.html 
# https://stackoverflow.com/questions/19094642/removing-long-horizontal-vertical-lines-from-edge-image-using-opencv
# function finds the outline of the arrow and filters out the horizontal and vertical lines so only the pointy part is left

def arrow_outline(img):
    dst = cv2.Canny(img, 50, 200, None, 3)
    
    # Copy edges to the images that will display the results in BGR
    cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)
    
    linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10) #probabilistic line transform
    final_pts = []
    
    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            
            thresh = 30
     
            if abs(l[1]-l[3]) < thresh: # aka vertical line
                pass
            elif abs(l[0]-l[2]) < thresh: # aka horizontal line
                pass
            else:
                start_pt = (l[0], l[1])
                end_pt = (l[2], l[3])
                final_pts.append(start_pt)
                final_pts.append(end_pt)
                cv2.line(img, start_pt, end_pt, (0,0,255), 3, cv2.LINE_AA)
    
    return img, final_pts

## Analyzing Data to Determine Arrow Direction

Displays text on annotated feed to tell us whether it sees left or right, if at all.

In [4]:
def arrow_direc(img, arrow_pts):
    
    if not arrow_pts:
        return 'NONE'
    
    else:
        arrow_coords = np.ravel(arrow_pts)
        x_list = arrow_coords[::2]
        y_list = arrow_coords[1::2]

        xmax = np.max(x_list)
        xmin = np.min(x_list)
        ymax = np.max(y_list)
        ymin = np.min(y_list)

#         if (xmax, ymax) in arrow_pts:
#             return 'LEFT'

        if (xmin, ymax) in arrow_pts:
            return 'RIGHT'

        else:
            return 'LEFT'

#     arrow_points = []
    
    
    
    
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     corners = cv2.goodFeaturesToTrack(gray,4,0.01,10)
#     corners = np.int0(corners)

#     ct='a'
#     for i in corners:
#         x,y = i.ravel()
# #         print(x,y)
#         cv2.circle(img,(x,y),3,(255,255,0),-1)
#         cv2.putText(img, ct, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA )
#         ct=ct+'a'

#     xmax, ymax = (np.max(corners, axis = 0)).ravel()
#     xmax_ind, ymax_ind = (np.argmax(corners, axis = 0)).ravel()
    
#     xmin, ymin = (np.min(corners, axis = 0)).ravel()
#     xmin_ind, ymin_ind = (np.argmin(corners, axis = 0)).ravel() 
    
#     if (xmax, ymax) in corners:
#         print("it's pointing left")
#         return 'LEFT'
        
#     if (xmin, ymax) in corners:
#         print("it's pointing right")
#         return 'DOWN'

#     else:
#         return 'IDK'

#     if(np.count_nonzero(corners[:,0,0] == xmax) == 2):
#         return'LEFT'
#     else:
#         return 'RIGHT'
    
    
    
#     if( abs(xmax-xmin) > abs(ymax-ymin)):
#         if(np.count_nonzero(corners[:,0,0] == xmax) == 2):
#             return 'LEFT'
#         else:
#             return 'RIGHT'
#     else:
#         if(np.count_nonzero(corners[:,0,1] == ymax) == 2):
#             return 'UP'
#         else:
#             return 'DOWN'   

## Colour Detection - Only for Blue

Annotates the feed with a blue bounding box to show where in the feed the colour blue is detected (if at all).

In [5]:
# https://www.geeksforgeeks.org/multiple-color-detection-in-real-time-using-python-opencv/

def blue_detec(imageFrame):
        # Convert the imageFrame in 
    # BGR(RGB color space) to 
    # HSV(hue-saturation-value)
    # color space
    hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)
  
    # Set range for blue color and
    # define mask
    blue_lower = np.array([94, 80, 2], np.uint8)
    blue_upper = np.array([120, 255, 255], np.uint8)
    blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)
      
    # Morphological Transform, Dilation
    # for each color and bitwise_and operator
    # between imageFrame and mask determines
    # to detect only that particular color
    kernel = np.ones((5, 5), "uint8")
      
    # For blue color
    blue_mask = cv2.dilate(blue_mask, kernel)
    res_blue = cv2.bitwise_and(imageFrame, imageFrame,
                               mask = blue_mask)

  
    # Creating contour to track blue color
    contours, hierarchy = cv2.findContours(blue_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
    box_dims = []
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 500):
            x, y, w, h = cv2.boundingRect(contour)
            imageFrame = cv2.rectangle(imageFrame, (x, y),
                                       (x + w, y + h),
                                       (255, 0, 0), 2)
            box_dims.append((x, y, w, h))

    return imageFrame, box_dims

## Main Function - Opens Up Video Feed and Calls Functions

In [2]:
def main(argv):
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FPS, 1)     # set the frame rate

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    while True:
        ret, frame = cap.read()
        
        blue_area, blue_bbox_dims = blue_detec(frame)
        
        if len(blue_bbox_dims) != 0:
            blue = True
        else: 
            blue = False
            
        arrow_finding, arrow_pts = arrow_outline(frame)
        
#         direction_finding = arrow_direc(arrow_finding, blue_bbox_dims, arrow_pts)
        
        direc = arrow_direc(arrow_finding, arrow_pts)
        
        org = (20, 20)
        
        if blue is True and direc !='NONE' and direc !='IDK':
            cv2.putText(arrow_finding, direc, org, cv2.FONT_HERSHEY_SIMPLEX, fontScale = 1,color = (0, 0, 0), 
                        thickness = 2 )

        cv2.imshow('Annotated Feed', frame)
        
#         if direction_finding != "inconclusive":
#             print("direction %s" % direction_finding)

        c = cv2.waitKey(1000) # adds delay in feed so we can read 
        if c == 27: # this is the esc key
            break

    cap.release()
    cv2.destroyAllWindows()

## Finally, run this! This calls the main function and starts the magic

In [ ]:
if __name__ == "__main__":
    main(sys.argv[1:])

In [55]:
# checks whether a point is within a rectangle with given - not used!

def solve(bl, tr, p) :
    if (p[0] > bl[0] and p[0] < tr[0] and p[1] > bl[1] and p[1] < tr[1]) :
        return True
    else :
        return False